In [1]:
import sys

sys.path.append("../")

In [2]:
import torch
from torch.utils.data import DataLoader

import hydra

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

from src.models.core_models.zero_shot_ris import ZeroShotRIS
from src.data.core_datasets.phrasecutdataset import PhraseCutDataset

In [3]:
with hydra.initialize(version_base="1.3", config_path="../conf"):
    cfg = hydra.compose(config_name="config.yaml")
cfg

{'CUDNN_BENCHMARK': False, 'DATALOADER': {'ASPECT_RATIO_GROUPING': True, 'FILTER_EMPTY_ANNOTATIONS': True, 'NUM_WORKERS': 4, 'REPEAT_THRESHOLD': 0.0, 'SAMPLER_TRAIN': 'TrainingSampler'}, 'DATASETS': {'PRECOMPUTED_PROPOSAL_TOPK_TEST': 1000, 'PRECOMPUTED_PROPOSAL_TOPK_TRAIN': 2000, 'PROPOSAL_FILES_TEST': [], 'PROPOSAL_FILES_TRAIN': [], 'TEST': ['coco_2017_val'], 'TRAIN': ['coco_2017_train_unlabeled_densecl_r101']}, 'GLOBAL': {'HACK': 1.0}, 'INPUT': {'CROP': {'ENABLED': False, 'SIZE': [0.9, 0.9], 'TYPE': 'relative_range'}, 'FORMAT': 'RGB', 'MASK_FORMAT': 'bitmask', 'MAX_SIZE_TEST': 1333, 'MAX_SIZE_TRAIN': 1333, 'MIN_SIZE_TEST': 800, 'MIN_SIZE_TRAIN': [640, 672, 704, 736, 768, 800], 'MIN_SIZE_TRAIN_SAMPLING': 'choice', 'RANDOM_FLIP': 'horizontal'}, 'MODEL': {'ANCHOR_GENERATOR': {'ANGLES': [[-90, 0, 90]], 'ASPECT_RATIOS': [[0.5, 1.0, 2.0]], 'NAME': 'DefaultAnchorGenerator', 'OFFSET': 0.0, 'SIZES': [[32, 64, 128, 256, 512]]}, 'BACKBONE': {'FREEZE_AT': 0, 'NAME': 'build_resnet_fpn_backbone'},

In [4]:
ZeroShotRIS?

Init signature: ZeroShotRIS(clip_pretrained_path, solo_config, solo_state_dict_path) -> None
Docstring:     
Base class for all neural network modules.

Your models should also subclass this class.

Modules can also contain other Modules, allowing to nest them in
a tree structure. You can assign the submodules as regular attributes::

    import torch.nn as nn
    import torch.nn.functional as F

    class Model(nn.Module):
        def __init__(self):
            super().__init__()
            self.conv1 = nn.Conv2d(1, 20, 5)
            self.conv2 = nn.Conv2d(20, 20, 5)

        def forward(self, x):
            x = F.relu(self.conv1(x))
            return F.relu(self.conv2(x))

Submodules assigned in this way will be registered, and will have their
parameters converted too when you call :meth:`to`, etc.

.. note::
    As per the example above, an ``__init__()`` call to the parent class
    must be made before assignment on the child.

:ivar training: Boolean represents whether this m

In [5]:
pretrained_path = "openai/clip-vit-base-patch32"

In [6]:
PHRASECUT_ROOT="/run/media/maverick/Backup/datasets/phrasecut"

In [7]:
img_size = 224
img_mean= [0.48145466, 0.4578275, 0.40821073]
img_std= [0.26862954, 0.26130258, 0.27577711] 

In [8]:
transforms = A.Compose([    
    # A.Resize(height=img_size, width=img_size, interpolation=cv2.INTER_CUBIC),
    A.Normalize(mean=img_mean,std=img_std),
    ToTensorV2(transpose_mask=True)
])

In [9]:
ds = PhraseCutDataset(
    data_root=PHRASECUT_ROOT,
    task_json_path="filtered_tasks/refer_train.json",
    tokenizer_pretrained_path=pretrained_path,
    transforms=transforms,
    return_tensors="pt",
    prompt_method="fixed",
    neg_prob=0
)

In [10]:
dl = DataLoader(ds)

In [11]:
sample = next(iter(dl))
sample

{'image': tensor([[[[-0.2594, -0.5660, -0.8580,  ..., -0.0405, -0.2448, -0.0550],
           [-0.2448, -0.5222, -0.7850,  ..., -0.1280, -0.2740, -0.1280],
           [-0.2594, -0.4930, -0.7412,  ..., -0.6974, -0.2886, -0.0842],
           ...,
           [-0.6244, -0.5806, -0.6098,  ..., -0.2302, -0.2156, -0.2302],
           [-0.6244, -0.5952, -0.6390,  ..., -0.2156, -0.2010, -0.2010],
           [-0.6098, -0.6098, -0.6682,  ..., -0.1280, -0.1134, -0.0842]],
 
          [[-0.7166, -0.9717, -1.1818,  ..., -1.3169, -1.0017, -0.8967],
           [-0.7016, -0.9267, -1.1218,  ..., -0.9867, -1.0767, -1.3319],
           [-0.7166, -0.8967, -1.0767,  ..., -1.3769, -0.8066, -0.5365],
           ...,
           [-1.0317, -0.9867, -1.0167,  ..., -0.7916, -0.7766, -0.7916],
           [-1.0317, -1.0017, -1.0467,  ..., -0.7766, -0.7616, -0.7616],
           [-1.0167, -1.0167, -1.0767,  ..., -0.6865, -0.6715, -0.6415]],
 
          [[-0.8688, -1.0394, -1.1816,  ..., -1.4518, -1.2385, -0.9967],
    

In [12]:
sample["image"].shape

torch.Size([1, 3, 300, 500])

In [13]:
sample["input_ids"].shape

torch.Size([1, 77])

In [14]:
image_input = sample["image"]
text_input = {
    "input_ids": sample["input_ids"].expand(2, -1),
    "attention_mask": sample["attention_mask"].expand(2, -1),
}

In [15]:
zsmodel = ZeroShotRIS(pretrained_path, cfg, "/run/media/maverick/Backup/checkpoints/FreeSOLO_R101_30k.pt")

In [16]:
zsmodel.eval()
with torch.inference_mode():
    output = zsmodel(image_input, text_input)

/home/maverick/Projects/ml-scratchpad/.venv/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Prediction Mask Shape: torch.Size([33, 300, 500])


RuntimeError: The size of tensor a (500) must match the size of tensor b (3) at non-singleton dimension 1

In [ ]:
output.shape

In [ ]:
from matplotlib import pyplot as plt

import numpy as np

In [ ]:
np_image = sample["image"][0].movedim(0, -1).numpy()

img_mean= np.array([0.48145466, 0.4578275, 0.40821073])
img_std= np.array([0.26862954, 0.26130258, 0.27577711])

norm_image = np_image * img_std + img_mean

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

ax1.imshow(norm_image)
ax2.imshow(output[0])

In [ ]:
def rescale_img(img):
    img_min = img.min((0, 1))
    return (img - img_min) / (img.max((0, 1)) - img_min)

In [ ]:
plt.imshow(sample["mask"].squeeze())